In [ ]:
import serial
import datetime
import mysql.connector

In [ ]:
# Configuração da conexão
config = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',
    'database': 'sin499-turbidimetro',
    'raise_on_warnings': True
}

In [ ]:
# Cria a conexão
cnx = mysql.connector.connect(**config)

# Cria um cursor para executar consultas
cursor = cnx.cursor()

In [ ]:
led = 13
ldr = 'A0'
valorldr = 0

# Inicialização da comunicação serial
serial_port = serial.Serial('COM3', 9600)

In [ ]:
# Função para ler o valor do sensor LDR
def read_ldr_value():
    serial_port.write(ldr.encode())
    response = serial_port.readline().decode().strip()
    if response.startswith('Valor LDR: '):
        value = int(response.replace('Valor LDR: ', ''))
        return value
    else:
        return None

In [ ]:
# Configuração inicial
serial_port.write(('pinMode(' + str(led) + ', OUTPUT)').encode())
serial_port.write(('pinMode(' + ldr + ', INPUT)').encode())

while True:
    valorldr = read_ldr_value()
    if valorldr is not None:
        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # Exemplo de inserção
        query = "INSERT INTO turbidimetro (sensoriamento_ldr, date_time) VALUES (%s, %s)"
        values = (valorldr, current_time)
        cursor.execute(query, values)
        # Faz o commit da transação
        cnx.commit()
        print(f"{current_time} - Valor LDR: {valorldr}")
    
    # Aciona o LED
    serial_port.write(('digitalWrite(' + str(led) + ', HIGH)').encode())

In [ ]:
# Fecha o cursor e a conexão
cursor.close()
cnx.close()